In [67]:
from dash import Dash, dcc, Output, Input  
import dash_bootstrap_components as dbc    
import plotly.express as px
import pandas as pd                       
import sqlite3
from urllib.request import urlopen
import json
import os


In [68]:
db = r'/Users/jennadodge/uofo-virt-data-pt-12-2021-u-b/Water_Quality_Analysis/Database/database.sqlite3'
# Connect to SQLite database
conn = sqlite3.connect(db)
  
# Create cursor object
cursor = conn.cursor()

contaminants_df = pd.read_sql_query("SELECT * FROM all_contaminants",conn)
# fips_df = pd.read_sql_query('SELECT * from FIPS_Codes', conn)

conn.close()

In [69]:
contaminants_df.head()

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor
0,VT,Ludlow VT,5001.0,Ludlow Village Water Department,2818,Bromodichloromethane,1.410,0.06,0.0,ppb,23.500000
1,VT,Ludlow VT,5001.0,Ludlow Village Water Department,2818,Chloroform,2.260,0.40,0.0,ppb,5.650000
2,VT,Ludlow VT,5001.0,Ludlow Village Water Department,2818,Dibromochloromethane,0.938,0.10,0.0,ppb,9.380000
3,VT,Ludlow VT,5001.0,Ludlow Village Water Department,2818,Nitrate,0.350,0.14,10.0,ppm,2.500000
4,VT,Ludlow VT,5001.0,Ludlow Village Water Department,2818,Total trihalomethanes (TTHMs)†,5.530,0.15,80.0,ppb,36.866667


In [70]:
contaminants_df.dtypes

State                    object
City                     object
Zip                     float64
Utility                  object
People_served             int64
Contaminant              object
Utility_Measurement     float64
EWG_Health_Guideline    float64
Legal_Limit             float64
Units                    object
Contaminant_Factor      float64
dtype: object

In [71]:
# Change Zip dtype to string
contaminants_df["Zip"] = contaminants_df["Zip"].astype(str)

In [72]:
# Get rid of the .0 at the end of zip
contaminants_df["Zip"] = contaminants_df["Zip"].str[:-2]

In [73]:
# add a leading zero
contaminants_df["Zip"] = contaminants_df["Zip"].apply('{:0>5}'.format)

In [74]:
# change dtype of Zip to string, get rid of .0 at the end, add a leading zero if needed
# contaminants_df["Zip"] = contaminants_df["Zip"].astype(str).str[:-2].apply('{:0>5}'.format)

In [75]:
contaminants_df.head()


,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor
0,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Bromodichloromethane,1.410,0.06,0.0,ppb,23.500000
1,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Chloroform,2.260,0.40,0.0,ppb,5.650000
2,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Dibromochloromethane,0.938,0.10,0.0,ppb,9.380000
3,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Nitrate,0.350,0.14,10.0,ppm,2.500000
4,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Total trihalomethanes (TTHMs)†,5.530,0.15,80.0,ppb,36.866667


In [76]:
contaminants_df['Zip'].value_counts()

93287    968
90630    756
64012    421
26559    381
26301    356
        ... 
49839      1
49834      1
93218      1
93254      1
18045      1
Name: Zip, Length: 2755, dtype: int64

In [77]:
# bring in zips to counties dataframe to merge
file_to_read = os.path.join("..","..","Census_Data_Cleaning","zips_to_counties.csv")
zips_to_counties = pd.read_csv(file_to_read,dtype={"zip": str})

In [78]:
zips_to_counties.dtypes

zip            object
county_fips     int64
state_id       object
dtype: object

In [79]:
zips_to_counties.head()

,zip,county_fips,state_id
0,00601,72001,PR
1,00602,72003,PR
2,00603,72005,PR
3,00606,72093,PR
4,00610,72011,PR


In [80]:
# Change county_fips datatype to string, add leading zero if needed
zips_to_counties["county_fips"] = zips_to_counties["county_fips"].astype(str).apply('{:0>5}'.format)

In [81]:
zips_to_counties.dtypes

zip            object
county_fips    object
state_id       object
dtype: object

In [82]:
df = pd.merge(contaminants_df,zips_to_counties,how="left",left_on="Zip",right_on='zip')

In [51]:
df.shape

(123658, 14)

In [52]:
df["county_fips"].nunique()

901

In [53]:
df.sample(30)

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor,zip,county_fips,state_id
70898,PA,Pittsburgh PA,15238,Blawnox Borough Water Department,2200,Dibromochloromethane,9.1100,0.100,0.0,ppb,91.100000,15238,42003,PA
71700,PA,Champion PA,15557,Seven Springs Municipal Authority,3000,Trichloroacetic acid,0.6170,0.100,0.0,ppb,6.170000,15557,42111,PA
69875,TX,Austin TX,76247,Hanby Acres,279,Barium,21.5000,700.000,2000.0,ppb,0.030714,76247,48121,TX
27637,MI,Holly MI,48430,Holly Village of,8557,Chloroform,3.1600,0.400,0.0,ppb,7.900000,48430,26093,MI
10786,MO,Montgomery City MO,63333,Montgomery City PWS,2712,Tribromoacetic acid,0.6630,0.040,0.0,ppb,16.575000,63333,29139,MO
59335,TX,Purdon TX,75102,Navarro Mills Water Supply Corporation,3744,Aluminum,26.3000,600.000,0.0,ppb,0.043833,75102,48349,TX
52766,WV,Philippi,26201,Philippi City Of,3269,Antimony,0.0333,1.000,6.0,ppb,0.033300,26201,54097,WV
49673,WV,Peterstown,24910,Red Sulphur Psd,5352,Barium,13.2000,700.000,2000.0,ppb,0.018857,24910,54025,WV
49639,WV,Gap Mills,24910,Gap Mills Psd,392,Radium combined (-226 &amp; -228),0.6600,0.050,5.0,pCi/L,13.200000,24910,54025,WV
102308,OR,Langlois,97406,Langlois Water District,220,Barium,75.7000,700.000,2000.0,ppb,0.108143,97406,41015,OR


In [105]:
df["Contaminant"].unique()

array(['Bromodichloromethane', 'Chloroform', 'Dibromochloromethane',
       'Nitrate', 'Total trihalomethanes (TTHMs)†',
       'Radium combined (-226 &amp; -228)', 'Bromoform', 'Manganese',
       'Barium', 'Dichloroacetic acid', 'Xylenes (total)', 'Toluene',
       'MTBE', 'Nitrate and nitrite', 'Haloacetic acids (HAA5)†',
       'Trichloroacetic acid', 'Radon', 'Perchlorate',
       'Hexachlorocyclopentadiene', 'Monochloroacetic acid',
       'Radium combined (-226 &amp; -228)*', 'Barium*', 'Chlorate*',
       'Chromium (hexavalent)*', 'Manganese*', 'Molybdenum*', 'Nitrate*',
       'Strontium*', 'Uranium', 'Vanadium*', 'Chromium (hexavalent)',
       'Chlorate', 'Strontium', 'Vanadium', 'Haloacetic acids (HAA9)†',
       'Di(2-ethylhexyl) phthalate', 'Molybdenum',
       'Haloacetic acids (HAA9)*†', '11-Dichloroethane*', '14-Dioxane*',
       'Monobromoacetic acid', 'Bromodichloromethane*', 'Chloroform*',
       'Dichloroacetic acid*', 'Haloacetic acids (HAA5)*†',
       'Total tri

In [24]:
# test out grouping by fips to determine top 10 contaminants
temp_df = df[df["county_fips"]=="41005"]

In [25]:
temp_df.head()

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor,zip,county_fips,state_id
101561,OR,Milwaukie OR,97004,City of Milwaukie,20500,Chromium (hexavalent),1.090,0.02,0.0,ppb,54.500000,97004,41005,OR
101562,OR,Milwaukie OR,97004,City of Milwaukie,20500,Haloacetic acids (HAA5)†,0.233,0.10,60.0,ppb,2.330000,97004,41005,OR
101563,OR,Milwaukie OR,97004,City of Milwaukie,20500,Haloacetic acids (HAA9)†,2.730,0.06,0.0,ppb,45.500000,97004,41005,OR
101564,OR,Milwaukie OR,97004,City of Milwaukie,20500,Nitrate,2.200,0.14,10.0,ppm,15.714286,97004,41005,OR
101565,OR,Milwaukie OR,97004,City of Milwaukie,20500,Nitrate and nitrite,2.000,0.14,10.0,ppm,14.285714,97004,41005,OR


In [102]:
temp_df.groupby(["Contaminant"]).head(20)

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor,zip,county_fips,state_id
101561,OR,Milwaukie OR,97004,City of Milwaukie,20500,Chromium (hexavalent),1.0900,0.02,0.0,ppb,54.500000,97004,41005,OR
101562,OR,Milwaukie OR,97004,City of Milwaukie,20500,Haloacetic acids (HAA5)†,0.2330,0.10,60.0,ppb,2.330000,97004,41005,OR
101563,OR,Milwaukie OR,97004,City of Milwaukie,20500,Haloacetic acids (HAA9)†,2.7300,0.06,0.0,ppb,45.500000,97004,41005,OR
101564,OR,Milwaukie OR,97004,City of Milwaukie,20500,Nitrate,2.2000,0.14,10.0,ppm,15.714286,97004,41005,OR
101565,OR,Milwaukie OR,97004,City of Milwaukie,20500,Nitrate and nitrite,2.0000,0.14,10.0,ppm,14.285714,97004,41005,OR
101566,OR,Milwaukie OR,97004,City of Milwaukie,20500,Total trihalomethanes (TTHMs)†,5.0300,0.15,80.0,ppb,33.533333,97004,41005,OR
101567,OR,Milwaukie OR,97004,City of Milwaukie,20500,14-Dioxane,0.0351,0.35,0.0,ppb,0.100286,97004,41005,OR
101568,OR,Milwaukie OR,97004,City of Milwaukie,20500,Barium,3.9400,700.00,2000.0,ppb,0.005629,97004,41005,OR
101569,OR,Milwaukie OR,97004,City of Milwaukie,20500,Manganese,0.8670,100.00,0.0,ppb,0.008670,97004,41005,OR
101570,OR,Milwaukie OR,97004,City of Milwaukie,20500,Strontium,0.0931,1500.00,0.0,ppb,0.000062,97004,41005,OR


In [26]:
temp_df=temp_df.groupby(["Contaminant"]).size().to_frame().sort_values([0],ascending = True).tail(10).reset_index()

In [27]:
temp_df.head(10)

,Contaminant,0
0,Chlorate,3
1,Nitrate,3
2,Nitrate and nitrite,3
3,Chromium (hexavalent),4
4,Haloacetic acids (HAA5)†,4
5,Haloacetic acids (HAA9)†,4
6,Manganese,4
7,Strontium,4
8,Total trihalomethanes (TTHMs)†,4
9,Vanadium,4


In [28]:
temp_df = temp_df.rename(columns={0:'Count of Contaminant'})

In [31]:
# df = pd.read_csv('../census_contaminant_priority_by_zip.csv', dtype={"zip":str,"fips":str})

In [62]:
df.head()

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor,zip,county_fips,state_id
0,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Bromodichloromethane,1.410,0.06,0.0,ppb,23.500000,05001,50027,VT
1,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Chloroform,2.260,0.40,0.0,ppb,5.650000,05001,50027,VT
2,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Dibromochloromethane,0.938,0.10,0.0,ppb,9.380000,05001,50027,VT
3,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Nitrate,0.350,0.14,10.0,ppm,2.500000,05001,50027,VT
4,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Total trihalomethanes (TTHMs)†,5.530,0.15,80.0,ppb,36.866667,05001,50027,VT


In [63]:
# get scatter plot contaminant algorithm
temp_df = df[df["county_fips"]=="41005"]

In [64]:
cont_sum_df = temp_df.groupby(by=["Contaminant"]).sum().sort_values(by=["Contaminant_Factor"], ascending=False)[['People_served',"Contaminant_Factor"]]

In [65]:
top_15_cont_sum = cont_sum_df.head(15)

In [66]:
top_15_cont_sum

,People_served,Contaminant_Factor
Contaminant,,
Haloacetic acids (HAA9)†,178243,1097.333333
Haloacetic acids (HAA5)†,178243,638.530000
Total trihalomethanes (TTHMs)†,178243,526.866667
Chromium (hexavalent),178243,60.345000
Nitrate,104311,19.370714
Nitrate and nitrite,104311,17.992857
Bromate,22729,12.900000
Vanadium,178243,0.544143
Chlorate,157743,0.524667


In [83]:
# scatter and hist plot algorithms
cont_fips_df = df

In [84]:
cont_fips_df.head()

,State,City,Zip,Utility,People_served,Contaminant,Utility_Measurement,EWG_Health_Guideline,Legal_Limit,Units,Contaminant_Factor,zip,county_fips,state_id
0,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Bromodichloromethane,1.410,0.06,0.0,ppb,23.500000,05001,50027,VT
1,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Chloroform,2.260,0.40,0.0,ppb,5.650000,05001,50027,VT
2,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Dibromochloromethane,0.938,0.10,0.0,ppb,9.380000,05001,50027,VT
3,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Nitrate,0.350,0.14,10.0,ppm,2.500000,05001,50027,VT
4,VT,Ludlow VT,05001,Ludlow Village Water Department,2818,Total trihalomethanes (TTHMs)†,5.530,0.15,80.0,ppb,36.866667,05001,50027,VT


In [85]:
cont_fips_df.shape

(123658, 14)

In [86]:
dff = cont_fips_df.copy()

In [87]:
dff = dff[dff['county_fips']=='41005']

In [93]:
dff2 = dff.groupby(["Contaminant"]).size().to_frame().sort_values([0], ascending = True).tail(10).reset_index()


In [94]:
dff2 = dff2.rename(columns={0: 'Count of Contaminant'})

In [95]:
dff2.shape

(10, 2)

In [96]:
dff2.head()

,Contaminant,Count of Contaminant
0,Chlorate,3
1,Nitrate,3
2,Nitrate and nitrite,3
3,Chromium (hexavalent),4
4,Haloacetic acids (HAA5)†,4


In [97]:
dff3 = dff.groupby(by=["Contaminant"]).sum().sort_values(by=['Contaminant_Factor'], ascending=False)[['People_served', 'Contaminant_Factor']]


In [98]:
top15_c_df = dff3.head(15)

In [99]:
top15_c_df

,People_served,Contaminant_Factor
Contaminant,,
Haloacetic acids (HAA9)†,178243,1097.333333
Haloacetic acids (HAA5)†,178243,638.530000
Total trihalomethanes (TTHMs)†,178243,526.866667
Chromium (hexavalent),178243,60.345000
Nitrate,104311,19.370714
Nitrate and nitrite,104311,17.992857
Bromate,22729,12.900000
Vanadium,178243,0.544143
Chlorate,157743,0.524667


In [100]:
top15_c_df = top15_c_df.reset_index()

In [101]:
top15_c_df

,Contaminant,People_served,Contaminant_Factor
0,Haloacetic acids (HAA9)†,178243,1097.333333
1,Haloacetic acids (HAA5)†,178243,638.530000
2,Total trihalomethanes (TTHMs)†,178243,526.866667
3,Chromium (hexavalent),178243,60.345000
4,Nitrate,104311,19.370714
5,Nitrate and nitrite,104311,17.992857
6,Bromate,22729,12.900000
7,Vanadium,178243,0.544143
8,Chlorate,157743,0.524667
9,Manganese,178243,0.199970
